In [1]:
import pandas as pd
import psycopg2
import os
import numpy as np

## 1. Creating Dataframes for phyla, classes, ..., genera and species from the WFO csv

In [2]:
columns_to_keep = ['taxonID',
                   'scientificName',
                   'taxonRank',
                   'parentNameUsageID',
                   'family',
                   'genus',
                   'nomenclaturalStatus',
                   'taxonomicStatus',
                   'acceptedNameUsageID'
                   ]

# WFO Database saved as csv file
input_csv = 'classification.csv'

# transforms taxonID to an integer ID (e.g wfo-0001302018 --> 1302018)
def taxon_number(taxon_id):
    if taxon_id[:3] == 'wfo':
        return int(taxon_id[4:])
    else:
        return 0

In [3]:
# list of english phyla names corresponding to the scientific names
phyla_english_names = [('Lycopodiophyta', 'Lycopods'),
                       ('Marchantiophyta', 'Liverworts'),
                       ('Angiosperms', 'Flowering plants'),
                       ('Anthocerotophyta', 'Hornworts'),
                       ('Bryophyta', 'Mosses'),
                       ('Polypodiophyta', 'Ferns'),
                       ('Pinophyta', 'Conifers and allies'),
                       ('Ginkgophyta', 'Ginkgo'),
                       ('Cycadophyta', 'Cycadophyta')
                       ]
phyla_english_names_df = pd.DataFrame(phyla_english_names, columns=['scientificName', 'phylum_englishName'])

In [4]:
wfo_df = pd.read_csv(input_csv,
                     sep='\t',
                     encoding='ANSI',
                     usecols=columns_to_keep,
                     converters={0: taxon_number, 5: taxon_number, 19: taxon_number}
                     )

#### 1.1 Exploring the data

In [5]:
# Possible values for nomenclaturalStatus and taxonomicStatus:
display(wfo_df['nomenclaturalStatus'].value_counts())
display(wfo_df['taxonomicStatus'].value_counts())
# Possible values for taxon rank:
# print(wfo_df['taxonRank'].value_counts())

nomenclaturalStatus
Valid           840842
Illegitimate     28364
Invalid          18175
Superfluous       1561
Conserved          521
Rejected           432
Name: count, dtype: int64

taxonomicStatus
Synonym      943255
Accepted     444907
Unchecked    187975
Name: count, dtype: int64

We will keep only Conserved and Valid nomenclaturalStatus, and Accepted taxonomicStatus. But first we save all the species which are synonyms of accepted species in a separate dataframe:

In [6]:
# species_synonyms_df = wfo_df.loc[(wfo_df['nomenclaturalStatus'] == 'Valid') & (wfo_df['taxonomicStatus'] == 'Synonym') & (wfo_df['taxonRank'] == 'species'), ['taxonID', 'scientificName', 'acceptedNameUsageID']]
# display(species_synonyms_df)

In [8]:
wfo_df = wfo_df[(wfo_df['nomenclaturalStatus'] == 'Valid') | (wfo_df['nomenclaturalStatus'] == 'Conserved')]
wfo_df = wfo_df[wfo_df['taxonomicStatus'] == 'Accepted']
wfo_df = wfo_df.sort_values('scientificName')
display(wfo_df)

,taxonID,scientificName,taxonRank,parentNameUsageID,family,genus,nomenclaturalStatus,taxonomicStatus,acceptedNameUsageID
1405743,4000000001,Aa,genus,7000000429,Orchidaceae,Aa,Valid,Accepted,0
316954,319089,Aa achalensis,species,4000000001,Orchidaceae,Aa,Valid,Accepted,0
756946,760991,Aa argyrolepis,species,4000000001,Orchidaceae,Aa,Valid,Accepted,0
918422,922666,Aa aurantiaca,species,4000000001,Orchidaceae,Aa,Valid,Accepted,0
923805,928062,Aa calceata,species,4000000001,Orchidaceae,Aa,Valid,Accepted,0
...,...,...,...,...,...,...,...,...,...
1442733,4000046085,Ã— Zygosepescalum,genus,7000000429,Orchidaceae,Zygosepescalum,Valid,Accepted,0
1441514,4000044858,Ã— Zygostylis,genus,7000000429,Orchidaceae,Zygostylis,Valid,Accepted,0
1443289,4000046644,Ã— Zygotoria,genus,7000000429,Orchidaceae,Zygotoria,Valid,Accepted,0
1443330,4000046685,Ã— Zygowarrea,genus,7000000429,Orchidaceae,Zygowarrea,Valid,Accepted,0


Now that we reduced the dataframe to valid or conserved entries, we list the possible taxonRank in taxonomic order:
<br/><br/>

|             | Any   | Valid  | Conserved|
|-------------|-------|--------| ----------- |
| kingdom     | 1     | 1      | 0 |
| subkingdom  | 2     | 2      | 0 |
| phylum      | 9     | 9      | 0 |
| class       | 16    | 16     | 0 |
| subclass    | 26    | 26     | 0 |
| superorder  | 2     | 2      | 0 |
| order       | 142   | 141    | 1 |
| family      | 717   | 698    | 19 |
| subfamily   | 90    | 89     | 1 |
| tribe       | 84    | 84     | 0 |
| subtribe    | 170   | 170    | 0 |
| genus       | 17910 | 17598  | 312 |
| subgenus    | 71    | 71     | 0 |
| section     | 267   | 267    | 0 |
| subsection  | 86    | 86     | 0 |
| series      | 22    | 22     | 0 |
| subseries   | 2     | 2      | 0 |
| species     | 377223 | 377164 | 59 |
| prole       | 1     |        |   |
| subspecies  | 23371 |        |   |
| variety     | 23821 |        |   |
| subvariety  | 3     |        |   |
| form        | 857   |        |   |

In [9]:
wfo_df_grouped_by_taxonRank = wfo_df.groupby('taxonRank')

In [10]:
parent_df = wfo_df.filter(['taxonID', 'scientificName', 'taxonRank', 'nomenclaturalStatus', 'taxonomicStatus', 'acceptedNameUsageID']).rename(columns={'taxonID': 'parentNameUsageID', 'scientificName': 'parentScientificName', 'taxonRank': 'parentTaxonRank', 'nomenclaturalStatus': 'parentNomenclaturalStatus', 'taxonomicStatus': 'parentTaxonomicStatus', 'acceptedNameUsageID': 'parentAcceptedNameUsageID'}).set_index(['parentNameUsageID'])
species_df = wfo_df_grouped_by_taxonRank.get_group('species').join(parent_df, on=['parentNameUsageID'], how='left')
print(f'We have {len(species_df)} valid and accepted species.')
print(f'and {species_df['parentTaxonRank'].value_counts().sum()} of those have valid and accepted parents.')
print(f'{pd.isnull(species_df['genus']).value_counts().sum()} have a genus and {pd.isnull(species_df['family']).value_counts().sum()} a family listed in the genus and family columns.')

We have 377223 valid and accepted species.
and 377204 of those have valid and accepted parents.
377223 have a genus and 377223 a family listed in the genus and family columns.


#### 1.2 Creating the Dataframes

In [11]:
# ---- definition of category dataframe ----
hierarchy = ['phylum', 'class', 'subclass', 'superorder', 'order', 'family', 'subfamily', 'tribe', 'subtribe', 'genus', 'subgenus', 'section', 'subsection', 'series', 'subseries', 'species']
dfs = {category: pd.DataFrame() for category in hierarchy}

In [13]:
def create_dataframes(level):
    '''
    Creates dataframes for a given level in the hierarchy.
    e.g: create_dataframes('subclass') gives the dataframe containing all subclasses listed in the csv file, with two columns for each parent level. For 'subclass' those would be the four columns 'taxonID_parent_phylum', 'scientificName_parent_phylum', 'taxonID_parent_class', 'scientificName_parent_class'
    :param level: one of the strings listed in hierarchy
    :return: dataframe
    '''
    parents = {category: dfs[category] for category in hierarchy[:hierarchy.index(level)]}
    if parents:
        if level == 'species':
            level_dfs = [wfo_df_grouped_by_taxonRank.get_group(level).filter(['taxonID', 'scientificName', 'parentNameUsageID', 'genus', 'family']).merge(parents[key].filter(['taxonID', 'scientificName']), how='inner', left_on='parentNameUsageID', right_on='taxonID', suffixes=(None, f"_parent_{key}")) for key in parents]
        else:
            level_dfs = [wfo_df_grouped_by_taxonRank.get_group(level).filter(['taxonID', 'scientificName', 'parentNameUsageID']).merge(parents[key].filter(['taxonID', 'scientificName']), how='inner', left_on='parentNameUsageID', right_on='taxonID', suffixes=(None, f"_parent_{key}")) for key in parents]
        # The goal is to concatenate all dataframes from levels_dfs, to get one level_df with two columns for each parent level. To do that without converting the IDs from int to float and therefore losing some digits, we need to ensure that each DataFrame contains all the columns and fill the missing integer columns with 0 while maintaining int64 type.
        all_columns = set().union(*[el.columns for el in level_dfs])
        filled_dataframes = []
        for df in level_dfs:
            missing_columns = list(all_columns.difference(df.columns))
            df_filled = pd.concat([df, pd.DataFrame(0, index=df.index, columns=missing_columns, dtype='int64')], axis=1)
            filled_dataframes.append(df_filled)
        level_df = pd.concat(filled_dataframes, ignore_index=True).replace(0, None).dropna(axis=1, how='all')
        return level_df
    else:
        phylum_df = wfo_df_grouped_by_taxonRank.get_group('phylum').filter(['taxonID', 'scientificName']).merge(phyla_english_names_df, how='left', on='scientificName')
        return phylum_df

In [14]:
for element in hierarchy:
    dfs[element] = create_dataframes(element)
    print(element, '\n', dfs[element].count(), '\n')

phylum 
 taxonID               9
scientificName        9
phylum_englishName    9
dtype: int64 

class 
 taxonID                         16
scientificName                  16
parentNameUsageID               16
taxonID_parent_phylum           16
scientificName_parent_phylum    16
dtype: int64 

subclass 
 taxonID                        26
scientificName                 26
parentNameUsageID              26
taxonID_parent_class           26
scientificName_parent_class    26
dtype: int64 

superorder 
 taxonID                           2
scientificName                    2
parentNameUsageID                 2
scientificName_parent_subclass    2
taxonID_parent_subclass           2
dtype: int64 

order 
 taxonID                             142
scientificName                      142
parentNameUsageID                   142
taxonID_parent_phylum                64
scientificName_parent_phylum         64
scientificName_parent_superorder     14
taxonID_parent_superorder            14
scientificName

### 2. Inserting data from df into postgres

In [19]:
db_params = {
    'dbname': 'WhatPlant_DB',
    'user': 'postgres',
    'password': 'Deluchka5770!',
    'host': 'localhost',
    'port': '5432'
}

conn = psycopg2.connect(
        dbname=db_params['dbname'],
        user=db_params['user'],
        password=db_params['password'],
        host=db_params['host'],
        port=db_params['port']
    )
cur = conn.cursor()

In [22]:
insert_into_phyla_query = '''   INSERT INTO WhatPlant.phyla (phylum_id, phylum_latin_name, phylum_english_name) 
                                VALUES (%s, %s, %s); '''

insert_into_classes_query = ''' INSERT INTO WhatPlant.classes (class_id, class_latin_name, phylum_id) 
                                VALUES (%s, %s, %s); '''

insert_into_subclasses_query = ''' INSERT INTO WhatPlant.subclasses (subclass_id, subclass_latin_name, class_id) 
                                   VALUES (%s, %s, %s); '''

insert_into_superorders_query = ''' INSERT INTO WhatPlant.superorders (superorder_id, superorder_latin_name, subclass_id) 
                                   VALUES (%s, %s, %s); '''

insert_into_orders_query = '''  INSERT INTO WhatPlant.orders (order_id, order_latin_name, superorder_id, subclass_id, class_id, phylum_id)
                                VALUES (%s, %s, %s, %s, %s, %s); '''

insert_into_families_query = '''    INSERT INTO WhatPlant.families (family_id, family_latin_name, order_id)
                                    VALUES (%s, %s, %s); '''

insert_into_subfamilies_query = ''' INSERT INTO WhatPlant.subfamilies (subfamily_id, subfamily_latin_name, family_id)
                                    VALUES (%s, %s, %s); '''

insert_into_tribes_query = '''  INSERT INTO WhatPlant.tribes (tribe_id, tribe_latin_name, subfamily_id)
                                VALUES (%s, %s, %s); '''

insert_into_subtribes_query = '''   INSERT INTO WhatPlant.subtribes (subtribe_id, subtribe_latin_name, tribe_id)
                                    VALUES (%s, %s, %s); '''

insert_into_genera_query = '''  INSERT INTO WhatPlant.genera (genus_id, genus_latin_name, subtribe_id, tribe_id, subfamily_id, family_id)
                                VALUES (%s, %s, %s, %s, %s, %s); '''

insert_into_subgenera_query = '''   INSERT INTO WhatPlant.subgenera (subgenus_id, subgenus_latin_name, genus_id)
                                    VALUES (%s, %s, %s); '''

insert_into_sections_query = '''    INSERT INTO WhatPlant.sections (section_id, section_latin_name, subgenus_id, genus_id)
                                    VALUES (%s, %s, %s, %s); '''

insert_into_subsections_query = ''' INSERT INTO WhatPlant.subsections (subsection_id, subsection_latin_name, section_id, genus_id)
                                    VALUES (%s, %s, %s, %s); '''

insert_into_series_query = '''  INSERT INTO WhatPlant.series (series_id, series_latin_name, section_id, genus_id)
                                VALUES (%s, %s, %s, %s); '''

insert_into_subseries_query = '''   INSERT INTO WhatPlant.subseries (subseries_id, subseries_latin_name, series_id)
                                    VALUES (%s, %s, %s); '''

insert_into_species_query = ''' INSERT INTO WhatPlant.species (species_id, species_latin_name, subseries_id, series_id, subsection_id, section_id, subgenus_id, genus_id, genus_name, family_name)
                                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s); '''

insert_into_species_synonyms_query = '''    INSERT INTO WhatPlant.species_synonyms 
                                            (species_synonym_id, species_synonym_name, synonym_for)
                                            VALUES (%s, %s, %s); '''

In [23]:
for _, row in dfs['phylum'].iterrows():
    phylum_data = (row['taxonID'], row['scientificName'], row['phylum_englishName'])
    cur.execute(insert_into_phyla_query, phylum_data)

for _, row in dfs['class'].iterrows():
    class_data = (row['taxonID'], row['scientificName'], row['taxonID_parent_phylum'])
    cur.execute(insert_into_classes_query, class_data)

for _, row in dfs['subclass'].iterrows():
    subclass_data = (row['taxonID'], row['scientificName'], row['taxonID_parent_class'])
    cur.execute(insert_into_subclasses_query, subclass_data)

In [24]:
for _, row in dfs['superorder'].iterrows():
    superorder_data = (row['taxonID'], row['scientificName'], row['taxonID_parent_subclass'])
    cur.execute(insert_into_superorders_query, superorder_data)

for _, row in dfs['order'].iterrows():
    order_data = (row['taxonID'], row['scientificName'], row['taxonID_parent_superorder'], row['taxonID_parent_subclass'], row['taxonID_parent_class'], row['taxonID_parent_phylum'])
    cur.execute(insert_into_orders_query, order_data)

for _, row in dfs['family'].iterrows():
    family_data = (row['taxonID'], row['scientificName'], row['taxonID_parent_order'])
    cur.execute(insert_into_families_query, family_data)
    
for _, row in dfs['subfamily'].iterrows():
    subfamily_data = (row['taxonID'], row['scientificName'], row['taxonID_parent_family'])
    cur.execute(insert_into_subfamilies_query, subfamily_data)

for _, row in dfs['tribe'].iterrows():
    tribe_data = (row['taxonID'], row['scientificName'], row['taxonID_parent_subfamily'])
    cur.execute(insert_into_tribes_query, tribe_data)
    
for _, row in dfs['subtribe'].iterrows():
    subtribe_data = (row['taxonID'], row['scientificName'], row['taxonID_parent_tribe'])
    cur.execute(insert_into_subtribes_query, subtribe_data)
    
for _, row in dfs['genus'].iterrows():
    genus_data = (row['taxonID'], row['scientificName'], row['taxonID_parent_subtribe'], row['taxonID_parent_tribe'],  row['taxonID_parent_subfamily'], row['taxonID_parent_family'])
    cur.execute(insert_into_genera_query, genus_data)
    
for _, row in dfs['subgenus'].iterrows():
    subgenus_data = (row['taxonID'], row['scientificName'], row['taxonID_parent_genus'])
    cur.execute(insert_into_subgenera_query, subgenus_data)

for _, row in dfs['section'].iterrows():
    section_data = (row['taxonID'], row['scientificName'], row['taxonID_parent_subgenus'], row['taxonID_parent_genus'])
    cur.execute(insert_into_sections_query, section_data)
    
for _, row in dfs['subsection'].iterrows():
    subsection_data = (row['taxonID'], row['scientificName'], row['taxonID_parent_section'], row['taxonID_parent_genus'])
    cur.execute(insert_into_subsections_query, subsection_data)

for _, row in dfs['series'].iterrows():
    series_data = (row['taxonID'], row['scientificName'], row['taxonID_parent_section'], row['taxonID_parent_genus'])
    cur.execute(insert_into_series_query, series_data)

for _, row in dfs['subseries'].iterrows():
    subseries_data = (row['taxonID'], row['scientificName'], row['taxonID_parent_series'])
    cur.execute(insert_into_subseries_query, subseries_data)

In [25]:
for _, row in dfs['species'].iterrows():
    species_data = (row['taxonID'], row['scientificName'], row['taxonID_parent_subseries'], row['taxonID_parent_series'], row['taxonID_parent_subsection'], row['taxonID_parent_section'], row['taxonID_parent_subgenus'], row['taxonID_parent_genus'], row['genus'], row['family'])
    cur.execute(insert_into_species_query, species_data)

In [26]:
conn.commit()
cur.close()
conn.close()